In [1]:
from importlib import reload

In [2]:
sys.path.append(os.path.join(os.path.abspath('../..'), 'src'))
sys.path.append(os.path.join(os.path.abspath('../..'), 'data'))

In [58]:
import utils
import bill_proc_utils
import bill_sponsor_analysis_pipeline
import build_repr_report_pages

In [79]:
reload(bill_proc_utils)
reload(bill_sponsor_analysis_pipeline)
reload(build_repr_report_pages)

<module 'build_repr_report_pages' from '/home/immersinn/gits/ncga/src/build_repr_report_pages.py'>

In [17]:
main_repo_dir = utils.get_main_dir()

## Load Data

In [18]:
reprs_info, bill_info, sponsor_info = bill_sponsor_analysis_pipeline.main('2014')

In [19]:
reprs_info.head()

,District,Session,Chamber,Name,Party,Incombant,Label,BillCount
35,36,2014,S,"Fletcher L. Hartsell, Jr.",R,NA,F.Hartsell (R),69
93,44,2014,H,Rick Glazier,D,NA,R.Glazier (D),69
9,10,2014,S,Brent Jackson,R,NA,B.Jackson (R),62
47,48,2014,S,Tom Apodaca,R,NA,T.Apodaca (R),57
40,41,2014,S,Jeff Tarte,R,NA,J.Tarte (R),54


In [20]:
bill_info.head()

,Session,Chamber,Bill,Content,LongTitle,TableInfo,Keywords,Sponsors
0,2015E4,H,1,GENERAL ASSEMBLY OF NORTH CAROLINA FOURTH EXT...,A HOUSE RESOLUTION adopting the permanent rule...,"{'Referred to': '', 'Sponsors': 'Representativ...","[ADOPTED, GENERAL ASSEMBLY, RESOLUTIONS, SIMPL...",[Lewis]
1,2015E4,H,2,GENERAL ASSEMBLY OF NORTH CAROLINA FOURTH EXT...,A JOINT RESOLUTIOn providing for adjournment s...,"{'Referred to': '', 'Sponsors': 'Representativ...","[ADJOURNMENT, GENERAL ASSEMBLY, RESOLUTIONS, J...",[Lewis]
2,2015E4,H,3,GENERAL ASSEMBLY OF NORTH CAROLINA FOURTH EXT...,AN ACT to provide further REGULATORY RELIEF TO...,"{'Referred to': '', 'Short Title': 'Regulato...","[ADMINISTRATION DEPT., ADMINISTRATIVE CODE, AD...","[McGrady, Dixon]"
3,2015E4,H,4,GENERAL ASSEMBLY OF NORTH CAROLINA FOURTH EXT...,AN ACT directing the department of transportat...,"{'Referred to': '', 'Short Title': 'Terminat...","[BRIDGES, CONTRACTS, COUNTIES, INFRASTRUCTURE,...","[Cotham, Bradford, J.Moore]"
4,2015E4,H,5,GENERAL ASSEMBLY OF NORTH CAROLINA FOURTH EXT...,AN ACT to clarify the service area for communi...,"{'Referred to': '', 'Short Title': 'Municipa...","[COUNTIES, EDGECOMBE COUNTY, INFORMATION TECHN...",[S.Martin]


In [22]:
sponsor_info.head()

,BillID,Chamber,Name,SponsorID
0,0,H,Lewis,102
1,1,H,Lewis,102
2,2,H,McGrady,166
3,2,H,Dixon,53
4,3,H,Cotham,149


## Data for Individual Represenatatives

In [36]:
# These are our lookup tables
reprs_bills = sponsor_info.groupby('SponsorID')
bills_reprs = sponsor_info.groupby('BillID')

In [37]:
s_ixs = list(reprs_info.index)
s_ixs[:10]

[35, 93, 9, 47, 40, 117, 146, 28, 45, 137]

In [38]:
repr_id = s_ixs[0]

### List of Bills

In [52]:
repr_bills = list(reprs_bills.get_group(repr_id)['BillID'])
repr_bills[:5]

[1285, 1286, 1313, 1314, 1317]

### Subset of Bills

In [54]:
sub_bi = build_repr_report_pages.get_repr_bi(bill_info, repr_bills)

In [55]:
sub_bi.head()

,Session,Chamber,Bill,Title,Keywords,Sponsors
1285,2015,S,91,SOG Pilot Project Standards. (Public),"[PILOT PROGRAMS, PUBLIC, REPORTS, SCHOOL OF GO...",[Hartsell]
1286,2015,S,92,Codifier of Rules appointment. (Public),"[ADMIN. HEARINGS OFFICE, ADMINISTRATIVE CODE, ...",[Hartsell]
1313,2015,S,119,GSC Technical Corrections 2015. (Public),"[ADMIN. PROCEDURE OVERSIGHT COMM., ADMINISTRAT...",[Hartsell]
1314,2015,S,120,DOI License Processing Fees. (Public),"[CONTRACTS, FEES, INSURANCE, INSURANCE DEPT., ...",[Hartsell]
1317,2015,S,123,Uniform Fraudulent Transfer Act. (Public),"[CHAPTERED, COMMERCE, CORPORATIONS, FOR-PROFIT...",[Hartsell]


### Bill Co-sponsor Info

In [56]:
repr_cosponsors =  build_repr_report_pages.build_cos_df(repr_bills, repr_id,
                 reprs_info,
                 reprs_bills, bills_reprs)

In [57]:
repr_cosponsors.head()

,ID,Name,Count
15,35,"Fletcher L. Hartsell, Jr.",53
19,45,Warren Daniel,3
7,19,Floyd McKissick,3
13,30,Joyce Krawiec,3
20,29,Shirley Randleman,2


### Bill Keyword Info

In [72]:
repr_keywords = bill_proc_utils.build_keywords_df(sub_bi, cutoff=5)

In [73]:
repr_keywords.shape

(50, 3)

In [74]:
repr_keywords.head()

,Keyword,Count,Rank
12,PUBLIC,67,1
364,EMPLOYMENT,18,2
238,BOARDS,17,3
146,LOCAL GOVERNMENT,15,4
360,INSURANCE,15,5


### All

In [80]:
repr_data = build_repr_report_pages.get_repr_data(repr_id, bill_info, reprs_info, reprs_bills, bills_reprs)

In [81]:
repr_data.keys()

dict_keys(['BillKeywords', 'Cosponsors', 'Bills'])